In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BertModel, BertTokenizer
import torch
import re
import numpy as np
from Ampep.toxic_func import toxic_feature
from Ampep.amp_func import amp_feature
import random

INFER_BATCH_SIZE = 32


2023-11-07 13:37:48.826929: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-07 13:37:50.561931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 13:37:55.986541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

tokenizer = T5Tokenizer.from_pretrained('prot_t5_xl_bfd/')
model = T5ForConditionalGeneration.from_pretrained('prot_t5_xl_bfd/').to('cuda')


In [3]:
from Bio import pairwise2

def similarity_score(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1,seq2)
    lens = max(len(seq1), len(seq2))
    try:
        return  -alignments[0].score/lens
    except Exception as e:
        print(e)
        print(alignments)
        print(seq1)
        print(seq2)
        return 0

/geniusland/home/qufuchuan/anaconda3/envs/maeseq/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [4]:
# def estimate_gradient(z, q, beta, criterion, sigma=100):
#     z_dim = z.shape[1]
#     u = np.random.normal(0, sigma, size=(q, z_dim)).astype('float32')
#     u = torch.from_numpy(u / np.linalg.norm(u, axis=1, keepdims=True))

#     f_0 = criterion(z)
#     f_tmp = criterion(z + beta*u)
#     return torch.mean(z_dim * u * np.expand_dims(f_tmp - f_0, 1)/ beta, dim=0,
#                       keepdims=True).to(z.dtype)
def estimate_gradient(z, q, beta, criterion, sigma=100):
    z_dim = z.shape[1:]
    u = np.random.normal(0, sigma, size=(q, z_dim[0],z_dim[1])).astype('float32')
    u = torch.from_numpy(u / np.linalg.norm(u, axis=1, keepdims=True)).to(device='cuda')

    f_0 = criterion(z)
    f_tmp = criterion(z + beta*u)
    print('Loss now: %f'%(f_0[0]))
    # print(f_0)
    u = u.to(device='cpu')
    # print(u.device)
    return torch.mean(z_dim[1] * u * np.expand_dims(np.expand_dims(f_tmp - f_0, 1),1)/ beta, dim=0,
                      keepdims=True).to(dtype = z.dtype, device = 'cuda')

In [30]:
def model_encode(seq):
    mask_rate = 0.15
    len_seq = len(seq)
    mask_len = int(len_seq * mask_rate)
    mask_idx = random.sample(range(len_seq), mask_len)
    
    raw_input = list(seq)
    for i,idx in enumerate(mask_idx):
        raw_input[idx] = '<extra_id_%d>'%i
    raw_input = [' '.join(raw_input)]
    inputs = tokenizer(raw_input, return_tensors='pt')['input_ids'].to('cuda')
    r1 = model.encoder.forward(inputs)['last_hidden_state']
    return r1

# def model_decode(emb):
#     outputs = [0]
#     for i in range(0, 100):
#         out = model.decoder.forward(torch.tensor([outputs], device='cuda'), encoder_hidden_states= emb)
#         out = model.lm_head(out['last_hidden_state'])
#         out = torch.softmax(out, dim = -1)
#         out = torch.argmax(out, dim = -1)
#         # if out[0][i] < 3 or out[0][i] > 22
#         outputs.append(int(out[0][i]))
#         if out[0][-1] == 1:
#             break
#         if outputs[-1] < 3 or outputs[-1] > 22:
#             outputs[-1] = 23
#         if i >= int(emb.shape[1] * 1.2):
#             break
#     seq = tokenizer.decode(outputs[1:-1])
#     seq = ''.join(seq.split())
#     return seq
def model_decode(emb):
    outputs = torch.zeros((emb.shape[0],1),dtype = int).cuda()
    max_len = int(emb.shape[1] * 1.2)
    
    split_num = emb.shape[0] // INFER_BATCH_SIZE if emb.shape[0] > INFER_BATCH_SIZE else 1
    outputs = list(torch.chunk(outputs, split_num, dim = 0))
    emb = list(torch.chunk(emb, split_num, dim = 0))
    for i in range(0, max_len):
        for j in range(0, len(outputs)):
            out = model.decoder.forward(outputs[j], encoder_hidden_states= emb[j])
            out = model.lm_head(out['last_hidden_state'])
            out = torch.softmax(out, dim = -1)
            out = torch.argmax(out, dim = -1)
            outputs[j] = torch.cat((outputs[j], out[:,-1].unsqueeze(-1)),1)
        # if out[0][i] < 3 or out[0][i] > 22
#             outputs = torch.cat((outputs[j*INFER_BATCH_SIZE, min((j+1)*INFER_BATCH_SIZE, emb.shape[0])], 
#                               out[j*INFER_BATCH_SIZE, min((j+1)*INFER_BATCH_SIZE, emb.shape[0]),-1].unsqueeze(-1)),1)
    outputs = torch.concat(outputs, dim = 0)
#     for single in outputs:
#         print(single.shape)
    seq = []
    for single in outputs:
        end_idx = (torch.where(single == 1))
        if len(end_idx) == 0:
            end_idx = len(single)
        else:
            end_idx = end_idx[0][0]
        res = tokenizer.decode(single[1:end_idx])
        seq.append(res)
    
#     seq = [tokenizer.decode(single[1: torch.where(single==1)[0][0]]) 
#        for single in outputs]
    seq = [''.join(single.split()) for single in seq]
    return seq

In [6]:
def loss_function(z, model,origin_seq ,weight=1, score=None, constraints=[],
                  weight_constraint=False):

    res = []
    num = z.shape[0]
    seq = model_decode(z)
    for i,strseq in enumerate(seq):
#         seq = model_decode(torch.unsqueeze(z[i], dim=0))
        
        loss_property = score(strseq, origin_seq) if score else 0

        loss_constraint = 0
        for c in constraints:
            loss_constraint += c(strseq)

        loss =  (loss_property + loss_constraint*weight if weight_constraint else
            loss_property*weight + loss_constraint)
        res.append(loss)
    return np.array(res)

In [7]:
from functools import partial
import time
from tqdm import tqdm
def optimize(model, seq, q=100, base_lr=0.1, max_iter=1000, num_restarts=1,
             weight=0.1, beta=1, use_adam=False, early_stop=False, score=similarity_score,
             constraints=[amp_feature, toxic_feature], writer=None, run_str=None, results_dir='results',
             init_best={}, write_log=None, flip_weight=False):
    z_0 = model_encode(seq)     #获得序列的embedding
    print(seq)
    loss = partial(loss_function, model=model, origin_seq = seq,weight=weight, score=score,
                   constraints=[amp_feature, toxic_feature], weight_constraint=flip_weight)
    best = {'score': -np.inf, 'found': False, 'early_stop': False}
    best.update(init_best)
    # start_time = time()
    for k in range(num_restarts):
        if best['early_stop']:
            break
        z = z_0.clone()
        # traj_z, traj_loss = [z.clone().numpy()], [loss(z)]
        # adam = torch.optim.Adam([z], lr=base_lr)
        for i in (range(max_iter)):
            print('start itr %d'%i)
            # print('Begin calculating gradient')
            grad = estimate_gradient(z, q, beta, loss)  # 使用QMO计算离散梯度
            # print('Begin optim')
            if use_adam:
                z.grad = grad
                # adam.step() # 更新梯度
            else:
                lr = ((1 - i/max_iter)**0.5) * base_lr
                z -= grad * lr
            # z.clamp_(-1, 1)

            # traj_z.append(z.clone().numpy())
            # traj_loss.append(loss(z))

            # print('After one optim')
            mol = model_decode(z)[0]   # 将优化后的embedding还原为序列
            print('After optim ', mol)
            mol_score = score and -score(mol, seq)
            print('score is %f'%mol_score)
            print('AMP : %d, Toxic: %d'%(constraints[0](mol), constraints[1](mol)))
            # desc = write_log(writer, z, grad, z_0, mol[0], seq[0], i)
            if (score is None or mol_score > best['score']) and all(c(mol) == 0 for c in constraints):
                # best.update(desc)
                print('Bingo!')
                best.update(dict(step=i, z=z, z_0=z_0, seq=mol,
                                 score=mol_score, found=True, run=k,
                                  early_stop=early_stop))

                print(f'PASSED!')
                # np.savez(os.path.join(results_dir, run_str), **best)

                if early_stop:
                    break
            print()
        # np.savez(os.path.join(results_dir, f"TRAJ{k}_"+run_str),
        #             z=np.stack(traj_z), loss=np.array(traj_loss))
    if not best['found']:
        print('Search failed!')
    return best


In [31]:
seq = 'WFHHIFRGIVHVGKTIHRLVTG'
optimize(model,seq)

WFHHIFRGIVHVGKTIHRLVTG
start itr 0


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 seq = 'WFHHIFRGIVHVGKTIHRLVTG'                                                               │
│ ❱ 2 optimize(model,seq)                                                                          │
│   3                                                                                              │
│                                                                                                  │
│ in optimize:24                                                                                   │
│                                                                                                  │
│   21 │   │   for i in (range(max_iter)):                                                         │
│   22 │   │   │   print('start itr %d'%i)                                                         │
│   23 │   │   │   # print('Begin calculating gradient')                                           │
│ ❱ 24 │   │   │   grad = estimate_gradient(z, q, beta, loss)  # 使用QMO计算离散梯度               │
│   25 │   │   │   # print('Begin optim')                                                          │
│   26 │   │   │   if use_adam:                                                                    │
│   27 │   │   │   │   z.grad = grad                                                               │
│                                                                                                  │
│ in estimate_gradient:16                                                                          │
│                                                                                                  │
│   13 │   u = torch.from_numpy(u / np.linalg.norm(u, axis=1, keepdims=True)).to(device='cuda')    │
│   14 │                                                                                           │
│   15 │   f_0 = criterion(z)                                                                      │
│ ❱ 16 │   f_tmp = criterion(z + beta*u)                                                           │
│   17 │   print('Loss now: %f'%(f_0[0]))                                                          │
│   18 │   # print(f_0)                                                                            │
│   19 │   u = u.to(device='cpu')                                                                  │
│                                                                                                  │
│ in loss_function:6                                                                               │
│                                                                                                  │
│    3 │                                                                                           │
│    4 │   res = []                                                                                │
│    5 │   num = z.shape[0]                                                                        │
│ ❱  6 │   seq = model_decode(z)                                                                   │
│    7 │   for i,strseq in enumerate(seq):                                                         │
│    8 #         seq = model_decode(torch.unsqueeze(z[i], dim=0))                                  │
│    9                                                                                             │
│                                                                                                  │
│ in model_decode:42                                                                               │
│                                                                                                  │
│   39 │   emb = list(torch.chunk(emb, split_num, dim = 0))          

In [ ]:
seq = 'AKKVFKRLGIGAVLWVLTTG'
optimize(model,seq)